## PDF Document Loaders
- Load various kind of documents from the web and local files.
- Apply LLM to the documents for summarization and question answering.

### Project 1: Question Answering from PDF Document
- We will load the document from the local file and apply LLM to answer the questions.
- Lets use research paper published on the missuse of the health supplements for workout. 

In [1]:
!pip install pymupdf tiktoken 


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [5]:
from dotenv import load_dotenv

load_dotenv('../../.env')

True

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("rag-dataset/Summarize Large Documents or Text Using LLMs and LangChain _ by Ranjeet Tiwari _ Senior Architect - AI _ IITJ _ Medium.pdf")

docs = loader.load()

In [3]:
len(docs)

6

In [4]:
docs[0].metadata
print(docs[0].page_content)

Member-only story
Summarize Large Documents or
Text Using LLMs and LangChain
Ranjeet Tiwari | Senior Architect - AI | IITJ · Follow
4 min read · Jul 17, 2024
100
Summarizing long texts can be quite a challenge, but with LangChain and
Language Learning Model (LLM), it’s made simple. Imagine you’re reading a
lengthy book or a detailed report, and you need to condense it into a short,
easy-to-read summary.
LangChain(with LLM) provides several strategies to help you do just that.
Let’s dive into these strategies using real-world examples to make things
clearer.
LangChain
The “Stuff” Strategy
The simplest method is called the “stuff” strategy. If the entire text fits within the
LLM’s context window, you can directly input the raw text and get a summary.
For example, suppose you have a short article about climate change:
Input Text:
“Climate change refers to long-term shifts and alterations in temperature
and weather patterns, primarily due to human activities like burning fossil
fuels, defo

In [6]:
### Read the list of PDFs in the dir
import os

pdfs = []
for root, dirs, files in os.walk("rag-dataset"):
    # print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

In [7]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)

    print(temp)
    # break

[Document(metadata={'producer': 'pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'creator': 'LaTeX with acmart 2021/08/29 v1.79 Typesetting articles for the Association for Computing Machinery and hyperref 2020-05-15 v7.00e Hypertext links for LaTeX', 'creationdate': '2022-07-05T00:43:24+00:00', 'source': 'rag-dataset/and_empirical_survey_on_long_document_summarization.pdf', 'file_path': 'rag-dataset/and_empirical_survey_on_long_document_summarization.pdf', 'total_pages': 39, 'format': 'PDF 1.5', 'title': 'An Empirical Survey on Long Document Summarization: Datasets, Models and Metrics', 'author': '', 'subject': '-  Information systems  ->  Summarization.-  Computing methodologies  ->  Information extraction.', 'keywords': '', 'moddate': '2022-07-05T00:43:24+00:00', 'trapped': '', 'page': 0}, page_content='1\nAn Empirical Survey on Long Document Summarization:\nDatasets, Models and Metrics\nHUAN YEE KOH, Monash University, Australia\nJIAXIN JU, Monash Un

In [8]:
len(docs)

45

In [10]:
def format_docs(docs):
    return "\n\n".join([x.page_content for x in docs])


context = format_docs(docs)

In [11]:
docs[0]

Document(metadata={'producer': 'pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'creator': 'LaTeX with acmart 2021/08/29 v1.79 Typesetting articles for the Association for Computing Machinery and hyperref 2020-05-15 v7.00e Hypertext links for LaTeX', 'creationdate': '2022-07-05T00:43:24+00:00', 'source': 'rag-dataset/and_empirical_survey_on_long_document_summarization.pdf', 'file_path': 'rag-dataset/and_empirical_survey_on_long_document_summarization.pdf', 'total_pages': 39, 'format': 'PDF 1.5', 'title': 'An Empirical Survey on Long Document Summarization: Datasets, Models and Metrics', 'author': '', 'subject': '-  Information systems  ->  Summarization.-  Computing methodologies  ->  Information extraction.', 'keywords': '', 'moddate': '2022-07-05T00:43:24+00:00', 'trapped': '', 'page': 0}, page_content='1\nAn Empirical Survey on Long Document Summarization:\nDatasets, Models and Metrics\nHUAN YEE KOH, Monash University, Australia\nJIAXIN JU, Monash Uni

In [12]:
print(context)

1
An Empirical Survey on Long Document Summarization:
Datasets, Models and Metrics
HUAN YEE KOH, Monash University, Australia
JIAXIN JU, Monash University, Australia
MING LIU∗, Deakin University, Australia
SHIRUI PAN∗†, Monash University, Australia
Long documents such as academic articles and business reports have been the standard format to detail out
important issues and complicated subjects that require extra attention. An automatic summarization system
that can effectively condense long documents into short and concise texts to encapsulate the most important
information would thus be significant in aiding the reader’s comprehension. Recently, with the advent of neural
architectures, significant research efforts have been made to advance automatic text summarization systems,
and numerous studies on the challenges of extending these systems to the long document domain have emerged.
In this survey, we provide a comprehensive overview of the research on long document summarization and


In [13]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")


In [14]:
encoding.encode("congratulations"), encoding.encode("rqsqeft")

([542, 111291, 14571], [81, 31847, 80, 5276])

In [15]:
len(encoding.encode(docs[0].page_content))

813

In [16]:
len(encoding.encode(context))

39876

In [17]:
969*64

62016

In [47]:
### Question Answering using LLM
from langchain_ollama import ChatOllama

from langchain_core.prompts import (SystemMessagePromptTemplate, HumanMessagePromptTemplate,
                                    ChatPromptTemplate)



from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
model = 'llama3.2:3b'
model = 'llama3.1:8b'
model = 'phi4:latest'
#model = 'deepseek-r1:8b'
#model = 'deepseek-r1:14b'

llm = ChatOllama(base_url=base_url, model=model)


In [48]:
system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who answer user question based on the provided context. 
                                                    Do not answer in more than {words} words""")

prompt = """Answer user question based on the provided context ONLY! If you do not know the answer, just say "I don't know".
            ### Context:
            {context}

            ### Question:
            {question}

            ### Answer:"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

# template
# template.invoke({'context': context, 'question': "How to gain muscle mass?", 'words': 50})

qna_chain = template | llm | StrOutputParser()

In [49]:
qna_chain

ChatPromptTemplate(input_variables=['context', 'question', 'words'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['words'], input_types={}, partial_variables={}, template='You are helpful AI assistant who answer user question based on the provided context. \n                                                    Do not answer in more than {words} words'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer user question based on the provided context ONLY! If you do not know the answer, just say "I don\'t know".\n            ### Context:\n            {context}\n\n            ### Question:\n            {question}\n\n            ### Answer:'), additional_kwargs={})])
| ChatOllama(model='phi4:latest', base_url='http://localhost:11434')
| StrOutputParser()

In [50]:
response = qna_chain.invoke({'context': context, 'question': "¿como resumir documentos largos?", 'words': 50})
print(response)

Para resumir documentos largos, puedes utilizar técnicas de procesamiento de lenguaje natural (NLP) con el apoyo de modelos de lenguaje grandes (LLMs). A continuación se presentan algunas estrategias y herramientas para lograrlo:

1. **Estrategia "Stuff"**: 
   - Combina todos los fragmentos del documento en un solo texto largo.
   - Aplica una instrucción al modelo indicando que resuma el contenido, ideal para documentos de longitud media.

2. **Estrategia "Map-Reduce"**:
   - Divide el documento grande en partes más pequeñas (chunks).
   - Resumir cada parte por separado utilizando un modelo LLM.
   - Combinar los resultados parciales en un resumen final.
   - Esta técnica es útil para documentos muy extensos, ya que divide la carga y mejora el manejo del contexto.

3. **Estrategia "Refine"**:
   - Resumir primero el documento completo con una instrucción de resumen general.
   - Luego, refinar cada sección del resumen agregando más detalle o aclaraciones según sea necesario.
   - Út

In [51]:
response = qna_chain.invoke({'context': context, 'question': "¿cuales son las estrategias de resumir documentos?", 'words': 50})
print(response)

Las estrategias para resumir documentos mencionadas en el texto incluyen:

1. **Estrategia "Stuff"**: Esta técnica implica combinar todo el contenido de un documento o múltiples documentos en una sola solicitud a un modelo de lenguaje grande (LLM). Es adecuada para textos cortos donde la longitud no supere los límites del modelo.

2. **Estrategia "Map-Reduce"**: Consiste en dividir el texto original en partes más pequeñas, resumir cada parte por separado y luego integrar estos resúmenes parciales en un único documento completo. Esta estrategia es útil para manejar documentos largos que exceden los límites de longitud del modelo.

3. **Estrategia "Refine"**: Después de obtener un primer borrador del resumen, esta técnica aplica correcciones o mejoras iterativas al texto inicial para mejorar la calidad y precisión del resumen final.

Estas estrategias se aplican dependiendo de la longitud y complejidad del texto a resumir.


In [52]:
response = qna_chain.invoke({'context': context, 'question': "Que es reinforcement learning?", 'words': 50})
print(response)

Reinforcement Learning (RL) is a type of machine learning where an agent learns to make decisions by taking actions in an environment to maximize some notion of cumulative reward. Unlike supervised learning, RL does not require labeled input/output pairs and differs from unsupervised learning because it focuses on finding a balance between exploration (of uncharted territory) and exploitation (refining known strategies).

Here's how reinforcement learning typically works:

1. **Agent**: The learner or decision-maker.
2. **Environment**: Everything the agent interacts with.
3. **Action**: A set of choices that an agent can make.
4. **State**: The current situation returned by the environment.
5. **Reward**: Feedback from the environment based on the action taken.

The process involves:

- The agent observes the state of the environment.
- It selects and performs an action.
- The environment responds with a new state and provides a reward (or penalty).
- The goal is to learn a policy tha

### Project 2: PDF Document Summarization

In [53]:
system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who works as document summarizer. 
                                                   You must not hallucinate or provide any false information.""")

prompt = """Summarize the given context in {words}.
            ### Context:
            {context}

            ### Summary:"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

summary_chain = template | llm | StrOutputParser()

In [54]:
summary_chain

ChatPromptTemplate(input_variables=['context', 'words'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are helpful AI assistant who works as document summarizer. \n                                                   You must not hallucinate or provide any false information.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'words'], input_types={}, partial_variables={}, template='Summarize the given context in {words}.\n            ### Context:\n            {context}\n\n            ### Summary:'), additional_kwargs={})])
| ChatOllama(model='phi4:latest', base_url='http://localhost:11434')
| StrOutputParser()

In [55]:
response = summary_chain.invoke({'context': context, 'words': 50})
print(response)

The article, written by Ranjeet Tiwari, discusses how LangChain can be used to summarize large documents with Large Language Models (LLMs), highlighting three strategies: "stuff," "map-reduce," and "refine."

1. **"Stuff" Strategy**: This approach involves inputting the entire text into a language model in one go. It's effective for shorter texts but may not work well for longer documents due to token limits.

2. **"Map-Reduce" Strategy**: Longer documents are divided into smaller parts, each of which is processed individually by a language model (the map phase). The resulting summaries from these parts are then combined to form a comprehensive summary (the reduce phase).

3. **"Refine" Strategy**: This method involves iteratively refining the text or its summary through multiple passes with the language model until a desired outcome is achieved.

Tiwari emphasizes that choosing between these strategies depends on the document's length and complexity, allowing for clear and concise sum

In [56]:
response = summary_chain.invoke({'context': context, 'words': 500})
print(response)

The article by Ranjeet Tiwari discusses techniques for summarizing large documents using Large Language Models (LLMs) and LangChain. It outlines three main strategies: "stuff," "map-reduce," and "refine."

1. **Stuff Strategy**: This method involves feeding the entire text into an LLM, which is suitable for smaller texts that can be processed in one go due to token limitations.

2. **Map-Reduce Strategy**: For longer texts, this approach splits the document into manageable chunks using a tool like LangChain's "DocumentChunker." Each chunk is summarized individually with an LLM, and these summaries are then combined to form a comprehensive summary of the entire text.

3. **Refine Strategy**: This method involves iteratively summarizing a large text by first breaking it down into smaller parts. Each part is summarized and the summaries themselves are further condensed until a concise final summary is achieved.

The article emphasizes choosing an appropriate strategy based on the length a

### Project 3: Report Generation from PDF Document

Streamlit Tutorial: https://www.youtube.com/watch?v=hff2tHUzxJM&list=PLc2rvfiptPSSpZ99EnJbH5LjTJ_nOoSWW

In [57]:
response = qna_chain.invoke({'context': context, 
                             'question': "Provide a detailed report from the provided context. Write answer in Markdown.", 
                             'words': 2000})
print(response)

# Detailed Report on Summarizing Large Documents Using LLMs and LangChain

## Overview

This report provides an overview of techniques for summarizing large documents using Large Language Models (LLMs) and LangChain, as detailed by Ranjeet Tiwari. The focus is on three primary strategies: "stuff," "map-reduce," and "refine." Each strategy caters to different needs based on document length and complexity.

## Strategies for Summarization

### 1. Stuff Strategy
- **Description**: This approach involves feeding the entire text into an LLM without segmenting it.
- **Use Case**: Best suited for short documents where context loss is minimal.

### 2. Map-Reduce Strategy
- **Process**:
  - **Mapping Phase**: The document is divided into smaller chunks that fit within the model's token limit (e.g., ~300 tokens).
  - **Reduction Phase**: Each chunk is summarized individually, and these summaries are then combined to form a comprehensive summary.
- **Application**: Ideal for medium-length documen